# Loading Libreries

In [ ]:
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import udf, avg, col
from snowflake.snowpark.types import IntegerType, FloatType, StringType, BooleanType
import pandas as pd
import sys
sys.path.append('..')
from utilities.creds import Credentials

In [ ]:
from snowflake.snowpark import version
print(version.VERSION)

# Creating Session

In [ ]:
session = Session.builder.configs(Credentials().__dict__).create()

# Creating resources

In [ ]:
print(session.sql('USE DATABASE LEARNINGSNOWPARKDB').collect())
print(session.sql('USE WAREHOUSE LEARNINGSNOWPARKVW').collect())
print(session.sql('CREATE OR REPLACE SCHEMA FRAUDDEMO').collect())
print(session.sql('USE SCHEMA FRAUDDEMO').collect())
print(session.sql('DROP TABLE IF EXISTS ORDERS').collect())
print(session.sql('DROP TABLE IF EXISTS ORDERS_DETAILS').collect())
print(session.sql('DROP TABLE IF EXISTS ENRICHED_DATA').collect())
print(session.sql('DROP TABLE IF EXISTS FRAUD_DETECTION').collect())
print(session.sql('DROP TABLE IF EXISTS NEW_TRANSACTION_DATA').collect())
print(session.sql('USE SCHEMA FRAUDDEMO').collect())
print(session.sql('CREATE STAGE IF NOT EXISTS UDFSTAGE').collect())
print(session.sql('SELECT CURRENT_WAREHOUSE(), CURRENT_DATABASE(), CURRENT_SCHEMA()').collect())

## Loading local data

In [ ]:
#Create an internal stage in your snowflake account-Db-Schema
session.sql('CREATE STAGE if not exists fraud_data').collect()

# Upload the source file to the stage
putOrders = session.file.put("orders.csv.gz", "@fraud_data", auto_compress=False, overwrite=True)
putOrdDet = session.file.put("order_details.csv.gz", "@fraud_data", auto_compress=False, overwrite=True)

putOrders
putOrdDet

## Exploring Data

In [ ]:
df = pd.read_csv('orders.csv.gz')
df.head()
df.dtypes

In [ ]:
df = df.astype({"JOIN_KEY":str, "SHIPPING_ZIPCODE":str})
df.dtypes

## Creating table from local pandas DataFrame

In [ ]:
%%time
snowpark_df = session.write_pandas(df, 'ORDERS', auto_create_table=True)

In [ ]:
#While you can use a pandas dataframe to create a table in snowflake. It is always a good practice to have a 
#defined schema

# Define the schema for the Order_Details table
from snowflake.snowpark.types import StructType, StructField

dfOrdDetSchema = StructType(
    [
        StructField("TRNX_ID", StringType()),
        StructField("ITEM", StringType()),
        StructField("PRICE", FloatType()),
        StructField("QTY", IntegerType())
    ]
)

# Create a reader
dfReader = session.read.schema(dfOrdDetSchema)
# Create a df using the schema and data file
dfOrdDet = dfReader.option("skip_header", 1).csv("@fraud_data/order_details.csv.gz")

In [ ]:
%%time
# Write the dataframe in a table, this is super quick because the files are already in snowflake stage
dfOrdDet.write.mode("overwrite").saveAsTable("ORDER_DETAILS")

## Closing Connection

In [ ]:
session.close()